In [ ]:
from __future__ import annotations

import os
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep
import numpy as np
import pandas as pd
from scipy import interpolate
from tqdm import tqdm

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
def mxmy(sample):
    mY = int(sample.split("-")[-1])
    mX = int(sample.split("NMSSM_XToYHTo2W2BTo4Q2B_MX-")[1].split("_")[0])

    return (mX, mY)

In [ ]:
plot_dir = Path("../../../../plots/XHY/Limits/23Aug24")
plot_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
cards_dir = "/eos/uscms/store/user/rkansal/bbVV/cards/Apr11"
samples = os.listdir(cards_dir)

Load / process limits

In [ ]:
limits = {" 2.5": [], "16.0": [], "50.0": [], "84.0": [], "97.5": []}

for sample in tqdm(samples):
    limits_path = f"{cards_dir}/{sample}/AsymptoticLimits.txt"
    if os.path.exists(limits_path):
        mx, my = mxmy(sample)
        with open(limits_path) as f:
            lines = f.readlines()

        nums = 0
        for i in np.arange(len(lines) - 1, -1, -1):
            line = lines[i][:-1]
            for key in limits:
                start_str = f"Expected {key}%: r < "
                if line.startswith(start_str):
                    limits[key].append([mx, my, float(line.split(start_str)[1])])
                    nums += 1

            if nums == 5:
                break

for key in limits:
    limits[key] = np.array(limits[key])

In [ ]:
limit_dir = plot_dir / "limits"
limit_dir.mkdir(exist_ok=True)

for key, limit in limits.items():
    df = pd.DataFrame(limit, columns=["MX", "MY", "Limit (fb)"])
    df.to_csv(f"{limit_dir}/limits_{key}.csv")

Plot

In [ ]:
def scatter2d(arr, title, name):
    fig, ax = plt.subplots(figsize=(14, 12))
    mappable = plt.scatter(
        arr[:, 0],
        arr[:, 1],
        s=150,
        c=arr[:, 2],
        cmap="turbo",
        norm=matplotlib.colors.LogNorm(vmin=0.01, vmax=100),
    )
    plt.title(title)
    plt.xlabel(r"$m_X$ (GeV)")
    plt.ylabel(r"$m_Y$ (GeV)")
    plt.colorbar(mappable)
    plt.savefig(name, bbox_inches="tight")

In [ ]:
def colormesh(xx, yy, lims, label, name):
    fig, ax = plt.subplots(figsize=(12, 8))
    _ = plt.pcolormesh(
        xx, yy, lims, norm=matplotlib.colors.LogNorm(vmin=0.05, vmax=1e4), cmap="turbo"
    )
    # plt.title(title)
    plt.xlabel(r"$m_X$ (GeV)")
    plt.ylabel(r"$m_Y$ (GeV)")
    plt.colorbar(label=label)
    hep.cms.label("Work in Progress", data=True, lumi="138", ax=ax)
    plt.savefig(name, bbox_inches="tight")

In [ ]:
mxs = np.logspace(np.log10(600), np.log10(3999), 100, base=10)
mys = np.logspace(np.log10(60), np.log10(250), 100, base=10)

xx, yy = np.meshgrid(mxs, mys)

In [ ]:
interpolated = {}
grids = {}

for key, val in limits.items():
    interpolated[key] = interpolate.LinearNDInterpolator(val[:, :2], np.log(val[:, 2]))
    grids[key] = np.exp(interpolated[key](xx, yy))

In [ ]:
for key, grid in grids.items():
    label = (
        f"{key}% expected exclusion limits (fb)"
        if key != "50.0"
        else "Median expected exclusion limits (fb)"
    )
    colormesh(xx, yy, grid, label, f"{plot_dir}/mesh_{key}_turbo.pdf")

In [ ]:
key = "50.0"
val = limits[key]
scatter2d(val, f"Expected {key}% Limit", f"{plot_dir}/scatter_{key}.pdf")